# Imports

In [ ]:
! pip install -U tensorflow-text==2.8.2
# pip install -U tensorflow_transform==1.8.0
# ! pip install mlflow

In [ ]:
!pip install wandb
import os
# 46c39bbac8b187eeb815386597ca25ba414d4c5e
!WANDB_API_KEY=$46c39bbac8b187eeb815386597ca25ba414d4c5e
os.environ['WANDB_API_KEY'] = '46c39bbac8b187eeb815386597ca25ba414d4c5e'
import wandb

from wandb.keras import WandbCallback


In [ ]:
import tensorflow_text as tf_text
# import tensorflow_transform as tft
import tensorflow as tf
# import unicodedata
import string
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, TextVectorization, Embedding, Flatten, Dropout, Softmax
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedShuffleSplit
seed=69

import matplotlib as mlp
import matplotlib.pyplot as plt

# mlp.rcParams['figure.figsize'] = (15, 10)

## returns a dictionary with 10 color values
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

from datetime import datetime
from pprint import pprint
import subprocess
from keras.callbacks import CSVLogger

# import mlflow
# mlflow.tensorflow.autolog()

# Utils

In [ ]:
def runcmd(cmd, verbose = False, *args, **kwargs):

    process = subprocess.Popen(
        cmd,
        stdout = subprocess.PIPE,
        stderr = subprocess.PIPE,
        text = True,
        shell = True
    )
    std_out, std_err = process.communicate()
    if verbose:
        print(std_out.strip(), std_err)
    pass

In [ ]:
def getClassWeights(y):
  'pass array of labels and get class weights dictionary'  
  neg, pos = np.bincount(y.reshape(-1))
  total = neg + pos
  weight_for_0 = (1 / neg) * (total / 2.0)
  weight_for_1 = (1 / pos) * (total / 2.0)
  class_weight = {0: weight_for_0, 1: weight_for_1} 

  return class_weight

In [ ]:
def getBatchedDataset(data=None, X=None, y=None, batch_size=64,):
  'pass df or X and y and get shuffled and batched dataset'
  if(type(data) == pd.core.frame.DataFrame):
    X = data.iloc[:,:-1].values
    y = data.iloc[:,-1:].values
  elif(type(data)==type(None) and type(X)==type(None) and type(y)==type(None)):
    print("pass dataset or X and y as argument to function")

  dataset = tf.data.Dataset.from_tensor_slices((X, y))
  
  batch_size = batch_size
  AUTOTUNE = tf.data.experimental.AUTOTUNE
  dataset = dataset.shuffle(buffer_size=len(dataset))\
              .batch(batch_size)\
              .cache()\
              .prefetch(AUTOTUNE)

  return dataset

In [ ]:
def getValData(frac=0.3, testdata_id="1DHHBfPgoTGEDltyw_-ox7JBI8pdyKiMm", random_state=69):
  'sample test set and get val data. returns dataframe'
  testdata_id = testdata_id
  path = "test_labelled.xlsx"
  if not (os.path.isfile(path)):
    ! gdown -q $testdata_id

  test = pd.read_excel("test_labelled.xlsx", usecols=[1,2])
  val_data = test.groupby('label', group_keys=False).apply(lambda x: x.sample(frac=frac, random_state=random_state))
  return val_data

In [ ]:
def getCallbacks(i='', patience=10, checkpoint_filepath=None):
  csv_logger = CSVLogger(f'training.log{i}', separator=',', append=False)
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
  # log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
  # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
  
  runcmd(f"mkdir tmp{i}")

  if(checkpoint_filepath==None):
    print(f"setting tmp{i} as checkpoint path")
    checkpoint_filepath = f'tmp{i}/'
  else : print(checkpoint_filepath)
    
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_filepath,
  save_weights_only=True,
  monitor='val_loss',
  mode='min',
  save_best_only=True)

  return [csv_logger, stop_early, model_checkpoint_callback]

In [ ]:
def getTrainData(trainid = "1BdFMY_cdiPtxBXVWjNxryqArpKEVETIB", return_dataframe=False):
  'pass id of drive file (default=train data) and get a tuple of comments and labels as numpy arrays'

  trainid = "1BdFMY_cdiPtxBXVWjNxryqArpKEVETIB"
  path = "13k_labelled.csv"
  if not (os.path.isfile(path)):
    runcmd(f"gdown -q {trainid}")

  data = pd.read_csv('13k_labelled.csv', index_col=None, header=None, names=['comment', 'label'])
  X = data.comment.values.reshape(-1,1)
  y = data.label.values.reshape(-1,1)

  if(return_dataframe == True):
    return data
    
  return X, y

# load data

In [ ]:
trainid = "1BdFMY_cdiPtxBXVWjNxryqArpKEVETIB"
X,y = getTrainData(trainid)
train_ds = tf.data.Dataset.from_tensor_slices((X, y))

In [ ]:
'train_text_ds = tf.data.Dataset.from_tensor_slices(tf.cast(X, tf.string))' # not req

'train_text_ds = tf.data.Dataset.from_tensor_slices(tf.cast(X, tf.string))'

# Build model layers

### custom preprocessing

In [ ]:
## apparently only bottom decorator works. both shouldddd


# @tf.keras.utils.register_keras_serializable(package='Custom', name="tf_normalize")
@tf.function
def tf_normalize(text):

  # print("testingggggg")

  # Split accecented characters.
  text = tf_text.normalize_utf8(text, 'NFKD')
  ## text = text.map(lambda line: tf.py_function(tf_text.normalize_utf8, [line], tf.string))

  text = tf.strings.lower(text)
  
  # remove special chars 
  # text = tf.strings.regex_replace(text, '[\\\!"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~]', '')  

  # Keep space, a to z only  (un-normalized unicode and emojis will get removed!).
  text = tf.strings.regex_replace(text, '[^a-z]', ' ')    # will replace all except alphabets with space, so 'bhench**d' becomes 'bhenc d', keeps spaces
  # text = tf.strings.regex_replace(text, '[^a-z]\s', '')   # will replace all special chars with '' , but keeps spaces too

  # remove repeated chars (better algo required)
  
  text = tf.strings.regex_replace(text, 'a{3,}', 'aa')
  text = tf.strings.regex_replace(text, 'b{3,}', 'b')
  text = tf.strings.regex_replace(text, 'c{3,}', 'c')
  text = tf.strings.regex_replace(text, 'd{3,}', 'dd')
  text = tf.strings.regex_replace(text, 'e{3,}', 'ee')
  text = tf.strings.regex_replace(text, 'f{3,}', 'f')
  text = tf.strings.regex_replace(text, 'g{3,}', 'g')
  text = tf.strings.regex_replace(text, 'h{3,}', 'h')
  text = tf.strings.regex_replace(text, 'i{3,}', 'i')
  text = tf.strings.regex_replace(text, 'j{3,}', 'j')
  text = tf.strings.regex_replace(text, 'k{3,}', 'k')
  text = tf.strings.regex_replace(text, 'l{3,}', 'l')
  text = tf.strings.regex_replace(text, 'm{3,}', 'm')
  text = tf.strings.regex_replace(text, 'o{3,}', 'oo')
  text = tf.strings.regex_replace(text, 'n{3,}', 'n')
  text = tf.strings.regex_replace(text, 'p{3,}', 'p')
  text = tf.strings.regex_replace(text, 'q{3,}', 'q')
  text = tf.strings.regex_replace(text, 'r{3,}', 'r')
  text = tf.strings.regex_replace(text, 's{3,}', 's')
  text = tf.strings.regex_replace(text, 't{3,}', 'tt')
  text = tf.strings.regex_replace(text, 'u{3,}', 'u')
  text = tf.strings.regex_replace(text, 'v{3,}', 'v')
  text = tf.strings.regex_replace(text, 'w{3,}', 'w')
  text = tf.strings.regex_replace(text, 'x{3,}', 'x')
  text = tf.strings.regex_replace(text, 'y{3,}', 'y')
  text = tf.strings.regex_replace(text, 'z{3,}', 'z')


  # strip and remove extra whitespace
  text = tf.strings.regex_replace(text, '\s\s+', ' ')
  text = tf.strings.strip(text)

  # remove all spaces
  # text = tf.strings.regex_replace(text, '\s+', '')

  text = tf.strings.join(['_', text, '_'], separator=' ')



  # Add spaces around punctuation.
  # text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')

  # text = tf.strings.join(['[START]', text, '[END]'], separator=' ')

  return text

In [ ]:
# tf.keras.utils.get_custom_objects()

In [ ]:
tf_normalize("23635476576874 hey there bitcchhh65$^%&*^%")

<tf.Tensor: shape=(), dtype=string, numpy=b'_ hey there bitcch _'>

### vectorize layer

In [ ]:
vocab_size = 30000
sequence_length = 200

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    standardize=tf_normalize,
    split='character',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length,
    pad_to_max_tokens=True,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False
)

In [ ]:
# config = vectorize_layer.get_config()
# config

In [ ]:
# load data for vocab
file_id = "19_b0lDdLoe5BlEsqvDdfc2AImSWRA5bo"
file_name = 'train_vocab_file.txt'
vocab_dir = "train_vocab"

if not os.path.exists(vocab_dir):  
  runcmd(f"mkdir {vocab_dir}")
if not os.path.exists(os.path.join(vocab_dir,file_name)) or os.path.exists(file_name):
  runcmd(f"gdown -q {file_id} -O {file_name}", verbose=False)
  runcmd(f"mv {file_name} {vocab_dir}")


In [ ]:
vocab_train_data = tf.keras.utils.text_dataset_from_directory(
    directory="train_vocab/",
    labels=None,
    label_mode=None,
    batch_size=32,
    max_length=None,
    shuffle=True,
    seed=69,
    validation_split=None,
    subset=None,
    follow_links=False
)

Found 1 files belonging to 1 classes.


In [ ]:
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
vectorize_layer.adapt(vocab_train_data)

In [ ]:
vocab = (vectorize_layer.get_vocabulary())
len(vocab)
vectorize_layer.vocabulary_size()

30

In [ ]:
# '_' in vocab 
# # vocab.index('_')
# sent = 'teri maa ki choooot'
# [vocab[i] for i in vectorize_layer([sent]).numpy().flatten() if i !=0]
# # lstm_layer1(embedding_layer(vectorize_layer(["66789975"])))


### embedding layer

In [ ]:
def getEmbeddingLayer(config):
  
  embedding_dim = config['embedding_dimensions']
  input_dim = vectorize_layer.vocabulary_size()
  input_length = sequence_length

  embedding_layer = Embedding(
      input_dim=input_dim,
      output_dim=embedding_dim,
      embeddings_initializer='uniform',
      mask_zero=True,
      input_length=input_length,
  )
  embedding_layer.embeddings_regularizer = tf.keras.regularizers.l2(1e-5)
  return embedding_layer
# input shape  -- (batch_size, input_length)
# output shape -- (batch_size, input_length, output_dim).

# Model val and train

## Utils for val_metrics and training and plotting

In [ ]:
METRICS = [
  keras.metrics.TruePositives(name='tp'),
  keras.metrics.FalsePositives(name='fp'),
  keras.metrics.TrueNegatives(name='tn'),
  keras.metrics.FalseNegatives(name='fn'), 
  keras.metrics.BinaryAccuracy(name='accuracy'),
  keras.metrics.Precision(name='precision'),
  keras.metrics.Recall(name='recall'),

  keras.metrics.AUC(name='auc', curve='ROC'),      
  # AUC refers to the Area Under the Curve of a Receiver Operating Characteristic curve (ROC-AUC). 
  # This metric is equal to the probability that a classifier will rank a random positive sample higher than a random negative sample.

  keras.metrics.AUC(name='prc', curve='PR'),
  # AUPRC refers to Area Under the Curve of the Precision-Recall Curve. 
  # This metric computes precision-recall pairs for different probability thresholds.
]

In [ ]:
def create_model(config=None):
  model = tf.keras.models.Sequential(name="rnn_model")
  model.add(vectorize_layer)
  model.add(getEmbeddingLayer(config))
  # model.add()
  model.add(Dropout(config['dropout_1']))

  lstm_layer1 = LSTM(
    units=config['lstm_units'], 
    activation='tanh',
    input_shape = (100,100),
    return_sequences=True,
    go_backwards = True,
  )
  model.add(lstm_layer1)
  model.add(Dropout(config['dropout_2']))

  # model.add(Dense(32, activation='tanh'))
  # model.add(Dropout(.2))
  model.add(Dense(config['dense_units'], activation='tanh'))
  model.add(Dropout(config['dropout_3']))
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  # opt1 = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.01, nesterov=True)
  opt2 = tf.keras.optimizers.Adam(config['learning_rate'])
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                optimizer=opt2,
              metrics=METRICS)
  return model


In [ ]:
def getValScores(X, y, config=None):
  '''
  arguments:
    model: tf model
    X:
    y:
    n: splits for StratifiedKfold
    epochs: in model.fit
    test_size: kfold, split testsize acts as val set for model.fit too
  '''
  n_splits = config['n_splits']
  epochs = config['epochs']
  test_size = config['test_size']

  sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=100)
  i=1
  histories = []
  # X,y = 
  val_scores = []
  

  for train_index,test_index in sss.split(X,y):

    run = wandb.init( project = "medusa", name=f'val-{i}', group="trial", job_type="cross-val 2", config=config, resume=False) 

    x_train,x_test=X[train_index],X[test_index]
    y_train,y_test=y[train_index],y[test_index]
    print(len(x_train), len(x_test), len(y_train[y_train==1]))
    # print(x_train, y_train)

    #class weights
    class_weights = getClassWeights(y_train)

    

    ## batch and shuffle
    train_ds = getBatchedDataset(X=x_train, y=y_train, batch_size=64)
    test_ds = getBatchedDataset(X=x_test, y=y_test, batch_size=64)  

    ## callbacks
    callbacks = getCallbacks(i)
    wandbcallback = WandbCallback(monitor='val_loss', mode='min', 
                                labels=["not profane", "profane"], 
                                save_model=False, 
                                # log_weights=True,
                                # log_gradients=True,
                                # training_data=(X,y), # nai horha...
                                # validation_data=(val_X,val_y), #
                                # save_weights_only=True #
                                )
    ## fit model
    model=create_model(config)
    history = model.fit(train_ds, epochs=epochs, validation_data=test_ds , 
                        class_weight=class_weights, callbacks=[*callbacks, wandbcallback], use_multiprocessing = True)

    histories.append(history)

    # model.save(f'model{i}')
    # tf.keras.models.save_model(model, f'model{i}')
    i=i+1
    val_score = model.evaluate(test_ds)
    val_scores.append(val_score)
    # print('Model evaluation ',val_score)

  # val_scores
  run.finish()

  return histories, val_scores



In [ ]:
# modelload = create_model(0.001)
# i=1
# checkpoint_filepath = f'tmp{i}/model_checkpoint'
# modelload.load_weights(checkpoint_filepath)

In [ ]:
def logDataset(run, make_dir=False ):

  if(make_dir==True):
    ! mkdir test
    ! mkdir train
    ! mv "13k_labelled.csv" "train/13k_labelled.csv"
    # ! mv "13k_labelled.csv" "train/13k_labelled.csv"
    
  train_data_artifact = wandb.Artifact('train-data', type='datasets', description='around 13k manually labelled comments each for training and testing')
  test_data_artifact = wandb.Artifact('train-data', type='datasets', description='around 13k manually labelled comments each for training and testing')
  train_data_artifact.add_dir('train')
  test_data_artifact.add_dir('test')
  run.log_artifact(train_data_artifact)
  run.log_artifact(test_data_artifact)


In [ ]:
def logPredictions(model, X, y, table_name):
  pred = model.predict(X);
  label_predictions = np.array([0 if x<0.5 else 1 for x in pred])
  df_dict = {'comments': X.flatten(),
             'true_label': y.flatten(),
             'label predictions': label_predictions.flatten(),
             'model probability': pred.flatten()}
  df = pd.DataFrame(df_dict)
  wandb.log({table_name: df})
  pass

In [ ]:
config = {
    'model':'RNN (lstm)',
    'architecture': 'V-EmDr-LDr-DDr-Fl-D',
    'tokenization':'character',
    'epochs': 50,
    'batch_size': 64,
    'optimizer': 'Adam',
    'learning_rate': 1e-3 ,
    'vect_sequence_len': 200,
    'embedding_dimensions': 120,
    'dropout_1': 0.3,
    'dropout_2': 0.4,
    'dropout_3': 0.4,
    'lstm_units': 16,
    'dense_units': 32,
}

In [ ]:
def trainModel(train_data=None, class_weights=None, val_data=None, log_predictions=False, log_model_checkpoint=False , log_data=False):
  'returns model and history'

  # run = wandb.init( project = "medusa", name='dataset', group="train-test", job_type="log data", resume=False) 
  # logDataset(run, make_dir=True)
  # if(config == None):
  run = wandb.init(config=config)
  
  epochs = config['epochs']
  batch_size = config['batch_size']

  # get val set sampled from test set and convert to batched dataset
  if(val_data is None):
    val_data = getValData()
    val_X = val_data.iloc[:,:-1].values
    val_y = val_data.iloc[:,-1:].values
    val_ds = getBatchedDataset(val_data, batch_size=batch_size)

  # get batched train dataset
  if(train_data==None):
    X,y = getTrainData()
    train_ds = getBatchedDataset(X=X,y=y, batch_size=batch_size)
  else:
    X = train_data.iloc[:,:-1].values
    y = train_data.iloc[:,-1:].values
    train_ds = getBatchedDataset(train_data, batch_size=batch_size)

  # class weights
  if(class_weights==None):
    class_weights = getClassWeights(y)

  #callbacks
  # reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.5, patience=3, min_lr=0.000001, verbose=1, cooldown=0)
  
  checkpoint_filepath = f'tmp/checkpoint/model_checkpoint'
  callbacks = getCallbacks('', patience=10, checkpoint_filepath=checkpoint_filepath)
  wandbcallback = WandbCallback(monitor='val_loss', mode='min', 
                                labels=["not profane", "profane"], 
                                save_model=False, 
                                log_weights=True,
                                log_gradients=True,
                                training_data=(X,y), # nai horha...
                                validation_data=(val_X,val_y), #
                                save_weights_only=True) #

  
  # train model
  model = create_model(config)
  history = model.fit(X,y, epochs=epochs, validation_data=(val_X,val_y) , 
                      class_weight=class_weights, 
                      callbacks=[*callbacks, wandbcallback],  #
                      use_multiprocessing = True)
  

  if(log_predictions==True):
    logPredictions(model, val_X, val_y, "val_data predictions")
    logPredictions(model, X, y, "train_data predictions")


  # wandb.log(history)
  if(log_model_checkpoint==True):
    trained_model_artifact = wandb.Artifact('lstm_model', type='model', description='lstm model with character tokenization')
    trained_model_artifact.add_dir('tmp/')
    run.log_artifact(trained_model_artifact)

  wandb.finish()
  
  return model, history

In [ ]:
## edit this function to make it general??

def plot_metrics(history, i):
  metrics = ['loss', 'precision', 'recall', 'auc', 'prc']
  plt.figure(figsize=(15,15))
  plt.suptitle(f'Model{i}')


  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(3,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[n], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[n], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([plt.ylim()[0]-.1, 1])
    elif metric == 'auc' or metric == 'prc':
      plt.ylim([plt.ylim()[0]-.1, 1])
    else:
      plt.ylim([plt.ylim()[0]-.1, plt.ylim()[1]+.1])

    plt.legend();
    plt.savefig(f'Model{i}_train_metrics.png') 

## Cross Validation

In [ ]:
val_config = dict (
    model= 'RNN (lstm)',
    architecture = 'V-EmDr-LDr-DDr-Fl-D',
    tokenization ='character',
    test_size  = 0.2,
    n_splits   = 1,
    epochs     = 1,
    batch_size = 64,
    optimizer  = 'Adam',
    learning_rate= 1e-3 ,
    vect_sequence_len= 200,
    embedding_dimensions= 120,
    dropout_1 = 0.3,
    dropout_2 = 0.4,
    dropout_3 = 0.4,
    lstm_units = 16,
    dense_units = 32,   
)

In [ ]:
X,y = getTrainData()
histories, val_scores = getValScores(X, y, config=val_config)

In [ ]:
val_metrics = pd.DataFrame(val_scores, index=range(1, 1+len(val_scores)), columns=['val-loss', 'tp', 'fp','tn', 'fn', 'accuracy', 'precision', 'recall', 'roc-auc', 'prc-auc'] )
print(val_metrics,"\n\n")
print(val_metrics.apply(np.mean, axis=0))

In [ ]:
## plot metrics in Kfold runs
for i, history in enumerate(histories):
  plot_metrics(histories[i],i+1)

## Train model 
and log metrics in wandb

In [ ]:
config = {
    'model':'RNN (lstm)',
    'architecture': 'V-EmDr-LDr-DDr-Fl-D',
    'tokenization':'character',
    'epochs': 50,
    'batch_size': 64,
    'optimizer': 'Adam',
    'learning_rate': 1e-3 ,
    'vect_sequence_len': 200,
    'embedding_dimensions': 120,
    'dropout_1': 0.3,
    'dropout_2': 0.4,
    'dropout_3': 0.4,
    'lstm_units': 16,
    'dense_units': 32,
}

In [ ]:
# wandb.config.update(config)

In [ ]:
# run = wandb.init()
# logDataset(run, make_dir=True)

In [ ]:
model, history = trainModel(config=config, log_predictions=True, log_model_checkpoint=True)

In [ ]:
wandb.finish()

## Hyperparam sweep

In [ ]:
sweep_config = {
  "name" : "trial-sweep",
  "method" : "random", 
}


In [ ]:
metric = {
    'name' : 'val_loss',
    'goal' : "minimize",
}
sweep_config['metric'] = metric

In [ ]:
sweep_config['early_terminate'] =  {
      'type': 'hyperband',
      'min_iter': 3,
}

In [ ]:
parameter_dict = {
    'model': { 
      'value': 'RNN (lstm)',
      # 'distribution': 'categorical'   
    },
    'tokenization':{
      'value':  'character',
      # 'distribution': 'categorical',
    },
    # 'architecture':{
    #   'values': 'V-EmDr-LDr-DDr-Fl-D',
    #   'distribution': 'categorical',
    # }, 
    "epochs" : {
      "values" : [15, 20, 25, 30, 50]
    },
    'optimizer': {
      'value':'Adam',
      # 'distribution': 'categorical'
    },
    "learning_rate" :{
      "distribution": 'log_uniform_values',
      # 'q': 1,
      "min": 0.00001,
      "max": 0.1
    },
    'embedding_dimensions': {
      # 'max': 240,
      # 'min': 60,
      # 'distribution': 'int_uniform'
      'values' : [50,100,150],
    },
    'dense_units': {
        'values':[16,32,64],
      # 'max': 64,
      # 'min': 16,
      # 'distribution': 'int_uniform',
    },
    'lstm_units': {
       'values': [8,16,32],
    #  'max': 32,
    #   'min': 8,
    #   'distribution': 'int_uniform'
  },
    'dropout_1': {
      # 'values': [0.1,0.2,0.3,0.4,0.5]
      'values': [0.4,0.5,0.6]
  },
    'dropout_2': {
    # 'values': [0.1,0.2,0.3,0.4,0.5]
    'values': [0.4,0.5,0.6]
  },
    'dropout_3': {
    # 'values': [0.1,0.2,0.3,0.4,0.5]
    'values': [0.4,0.5,0.6]
},
  
}

sweep_config["parameters"] = parameter_dict


In [ ]:
pprint(sweep_config)

Pretty printing has been turned OFF


In [ ]:
# sweep_id = wandb.sweep(sweep_config, project='medusa-sweeps')
# sweep_id = 'kgd9j04y'

NameError: ignored

In [ ]:
# sweep_id = 'hsb4t0em'
sweep_id = '0hljr5r1'  #curr
# wandb.init(sweep_config)
# wandb.finish()

In [ ]:
count = 10 # number of runs to execute
wandb.agent(sweep_id, function=trainModel, count=count, project='medusa-sweeps')

wandb: Agent Starting Run: hqj6mr3f with config:
wandb: 	dense_units: 16
wandb: 	dropout_1: 0.6
wandb: 	dropout_2: 0.5
wandb: 	dropout_3: 0.6
wandb: 	embedding_dimensions: 50
wandb: 	epochs: 30
wandb: 	learning_rate: 0.0001473231451411038
wandb: 	lstm_units: 32
wandb: 	model: RNN (lstm)
wandb: 	optimizer: Adam
wandb: 	tokenization: character


Epoch 1/50
416/417 [============================>.] - ETA: 0s - loss: 0.6353 - tp: 1633.0000 - fp: 5834.0000 - tn: 7874.0000 - fn: 636.0000 - accuracy: 0.5950 - precision: 0.2187 - recall: 0.7197 - auc: 0.7095 - prc: 0.2981

wandb: WARNING Non-numeric values found in layer: parameters/text_vectorization.weights, not logging this layer


417/417 [==============================] - 29s 57ms/step - loss: 0.6352 - tp: 1633.0000 - fp: 5837.0000 - tn: 7880.0000 - fn: 637.0000 - accuracy: 0.5950 - precision: 0.2186 - recall: 0.7194 - auc: 0.7094 - prc: 0.2978 - val_loss: 0.5371 - val_tp: 379.0000 - val_fp: 715.0000 - val_tn: 2475.0000 - val_fn: 303.0000 - val_accuracy: 0.7371 - val_precision: 0.3464 - val_recall: 0.5557 - val_auc: 0.7253 - val_prc: 0.3747
Epoch 2/50
417/417 [==============================] - 17s 41ms/step - loss: 0.5851 - tp: 1289.0000 - fp: 3353.0000 - tn: 8077.0000 - fn: 603.0000 - accuracy: 0.7030 - precision: 0.2777 - recall: 0.6813 - auc: 0.7609 - prc: 0.3714 - val_loss: 0.5909 - val_tp: 464.0000 - val_fp: 947.0000 - val_tn: 2243.0000 - val_fn: 218.0000 - val_accuracy: 0.6991 - val_precision: 0.3288 - val_recall: 0.6804 - val_auc: 0.7543 - val_prc: 0.4066
Epoch 3/50
417/417 [==============================] - 19s 45ms/step - loss: 0.5573 - tp: 1346.0000 - fp: 3218.0000 - tn: 8212.0000 - fn: 546.0000 - acc

accuracy,▁▃▄▄▅▅▆▆▇▇▇▇▇▇▇█████████
auc,▁▂▃▄▅▅▆▆▇▇▇▇▇▇██████████
epoch,▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇██
fn,█▇▇▆▅▅▄▄▄▄▃▃▂▂▂▂▂▁▂▁▁▁▁▁
fp,█▅▄▄▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▇▆▆▅▅▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
prc,▁▂▃▃▄▅▅▆▆▆▇▇▇▇▇█████████
precision,▁▂▂▃▃▄▄▅▅▅▆▆▇▇▇▇▇██▇████
recall,▂▁▂▃▃▄▄▅▅▅▆▆▆▆▇▇▇█▇█████
tn,▁▁▂▃▃▄▅▅▆▆▇▇▇▇▇▇████████
tp,▇▁▂▃▃▄▄▅▅▅▆▆▆▆▇▇▇█▇█████


...

mkdir: cannot create directory ‘tmp’: File exists
Epoch 1/50
417/417 [==============================] - 30s 60ms/step - loss: 0.6442 - tp: 1523.0000 - fp: 3891.0000 - tn: 10729.0000 - fn: 1051.0000 - accuracy: 0.7126 - precision: 0.2813 - recall: 0.5917 - auc: 0.7323 - prc: 0.4222 - val_loss: 0.4788 - val_tp: 279.0000 - val_fp: 383.0000 - val_tn: 2807.0000 - val_fn: 403.0000 - val_accuracy: 0.7970 - val_precision: 0.4215 - val_recall: 0.4091 - val_auc: 0.7383 - val_prc: 0.3878
Epoch 2/50
417/417 [==============================] - 19s 46ms/step - loss: 0.5868 - tp: 1335.0000 - fp: 3557.0000 - tn: 7873.0000 - fn: 557.0000 - accuracy: 0.6912 - precision: 0.2729 - recall: 0.7056 - auc: 0.7594 - prc: 0.3654 - val_loss: 0.5307 - val_tp: 414.0000 - val_fp: 724.0000 - val_tn: 2466.0000 - val_fn: 268.0000 - val_accuracy: 0.7438 - val_precision: 0.3638 - val_recall: 0.6070 - val_auc: 0.7569 - val_prc: 0.4141
Epoch 3/50
417/417 [==============================] - 18s 42ms/step - loss: 0.5577 - tp:

accuracy,▂▁▂▂▃▄▅▅▆▆▇▇▇▇▇▇▇███████████
auc,▁▂▃▃▄▅▆▆▆▇▇▇▇▇▇█████████████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
fn,█▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
fp,█▇▇▆▅▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
prc,▂▁▂▂▃▄▅▅▆▆▇▇▇▇▇▇▇█▇█████████
precision,▁▁▁▂▂▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████
recall,▁▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇█▇█████████
tn,█▁▂▂▃▄▅▆▆▆▇▇▇▇▇▇▇███████████
tp,▄▁▁▂▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇█████


wandb: Agent Starting Run: nafiay1k with config:
wandb: 	dense_units: 64
wandb: 	dropout_1: 0.5
wandb: 	dropout_2: 0.4
wandb: 	dropout_3: 0.5
wandb: 	embedding_dimensions: 100
wandb: 	epochs: 20
wandb: 	learning_rate: 7.945793746196646e-05
wandb: 	lstm_units: 32
wandb: 	model: RNN (lstm)
wandb: 	optimizer: Adam
wandb: 	tokenization: character


mkdir: cannot create directory ‘tmp’: File exists
Epoch 1/50
417/417 [==============================] - 29s 58ms/step - loss: 0.6343 - tp: 1804.0000 - fp: 4642.0000 - tn: 9978.0000 - fn: 770.0000 - accuracy: 0.6852 - precision: 0.2799 - recall: 0.7009 - auc: 0.7671 - prc: 0.4418 - val_loss: 0.5541 - val_tp: 413.0000 - val_fp: 754.0000 - val_tn: 2436.0000 - val_fn: 269.0000 - val_accuracy: 0.7358 - val_precision: 0.3539 - val_recall: 0.6056 - val_auc: 0.7439 - val_prc: 0.3686
Epoch 2/50
417/417 [==============================] - 17s 41ms/step - loss: 0.5822 - tp: 1318.0000 - fp: 3484.0000 - tn: 7946.0000 - fn: 574.0000 - accuracy: 0.6954 - precision: 0.2745 - recall: 0.6966 - auc: 0.7632 - prc: 0.3715 - val_loss: 0.5177 - val_tp: 391.0000 - val_fp: 650.0000 - val_tn: 2540.0000 - val_fn: 291.0000 - val_accuracy: 0.7570 - val_precision: 0.3756 - val_recall: 0.5733 - val_auc: 0.7585 - val_prc: 0.4176
Epoch 3/50
417/417 [==============================] - 19s 46ms/step - loss: 0.5550 - tp: 1

accuracy,▁▁▂▂▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇█▇██████████████
auc,▁▁▂▂▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇████████████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
fn,█▆▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
fp,█▆▆▅▅▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▇▇▆▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
prc,▂▁▂▂▃▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇██████████████
precision,▁▁▁▂▂▃▃▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇██▇██████
recall,▁▁▂▂▃▃▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████
tn,▆▁▂▂▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇████████████████
tp,█▁▁▂▂▃▃▄▄▄▄▅▅▅▆▅▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇


In [ ]:
# wandb.agent(sweep_id, function=my_train_func)

## plot model metrics

#### plot training loss for train and test data

In [ ]:
for i, history in enumerate(histories):
  plot_metrics(histories[i],i+1)

# Load and test model

## utils

In [ ]:
def loadModelFromCheckpoint(checkpoint_filepath=None, validate=False):
  'returns model with weights. also evaluates against val_data if validate=True '

  if(checkpoint_filepath==None):
    print("Pass a filepath to checkpoint_filepath")
    return
    
  model = create_model()
  model.load_weights(checkpoint_filepath)

  if validate==True:
    val_data = getValData()
    val_ds = getBatchedDataset(val_data, batch_size=64)
    print("Model Evaluation: \n", model.evaluate(val_ds))

  return model

In [ ]:
def loadModel(model_path=None, validate=False):

  if(model_path==None):
    print("Pass a filepath to model_path")
    return

  custom_objects = ({"tf_normalize": tf_normalize})
  with tf.keras.utils.custom_object_scope(custom_objects):
    load_model = tf.keras.models.load_model(model_path)

  if validate==True:
    val_data = getValData()
    val_ds = getBatchedDataset(val_data, batch_size=64)
    print("Model Evaluation: \n", model.evaluate(val_ds))
  
  return model

## load from drive

In [ ]:
# model = create_model()
# config = model.get_config()
# config

# custom_objects = ({"tf_normalize": tf_normalize})
# with tf.keras.utils.custom_object_scope(custom_objects):
#     new_model = tf.keras.Sequential().from_config(config)

'isse saved model load ni hoga: ye karke ig model weights load kar sakte hai'

In [ ]:
def folder_download(folder_id):
  # authenticate
  from google.colab import auth
  auth.authenticate_user()
  # get folder_name
  from googleapiclient.discovery import build
  service = build('drive', 'v3')
  folder_name = service.files().get(fileId=folder_id).execute()['name']
  # import library and download
  ! wget -qnc https://github.com/segnolin/google-drive-folder-downloader/raw/master/download.py
  from download import download_folder
  download_folder(service, folder_id, './', folder_name)
  return folder_name

In [ ]:
folder_download('1-kM4C1Ju8sW-VALL74fM48MgLmOehuRV')

In [ ]:
load_model = loadModel('char-tok-class-weights/model2', validate=True)

In [ ]:
# load_model.summary()
# tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
sample = np.array(['ksakas sdbfjkas knfalkclaskco dals jfjdd', 'oyee kaha se hai tu', 'choro sharma ke bete ko bolna padle', 'randi madarchod suno ji', '49213702194', 'chodke mat jao mujhe', 'chodke rakhdunga tujhe '])
print(sample.shape)
samplers = sample.reshape(-1,1)
# print(samplers, samplers.shape)

# print(load_model.predict(sample))
load_model.predict(samplers)

In [ ]:
# load_model.layers[3](load_model.layers[2](load_model.layers[1](load_model.layers[0](sample))))

## load artifact from wandb

In [ ]:
import wandb
run = wandb.init()

In [ ]:
artifact = run.use_artifact('mindadeepam/medusa/model-weights_only:v21', type='model')
# dir(artifact)

In [ ]:
artifact_dir = artifact.download()

In [ ]:
load_model = loadModel('artifacts/model-weights_only:v21')
model_ch = loadModelFromCheckpoint('artifacts/model-weights_only:v21')
### how can both be possible from same dir????

In [ ]:
load_model.predict([""])

array([[0.13954812]], dtype=float32)

In [ ]:
model_ch.predict([''])

array([[0.06186688]], dtype=float32)

In [ ]:
loadModelFromCheckpoint('tmp/model_checkpoint')